In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder
# from sklearn.preprocessing import OneHotEncoder

In [2]:
featured_df = pd.read_csv("featured_df.csv", encoding="utf-8")
featured_df.head()

,x,y,num_vowels,num_words,num_chars,num_digits,num_symbols,num_A,num_B,num_C,...,num_T,num_U,num_V,num_W,num_X,num_Y,num_Z,is_plural,has_char_repeat,has_color
0,MXCN SHRDDD CHS BLND,MEXICAN SHREDDED CHEESE BLEND,0,4,20,0,0,0,1,2,...,0,0,0,0,1,0,0,0,1,0
1,VGTBL SHRTNNG,VEGETABLE SHORTENING,0,2,13,0,0,0,1,0,...,2,0,1,0,0,0,0,0,1,0
2,SQD,SQUID,0,1,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,SLCD MNZNLL LVS SLD,SLICED MANZANILLA OLIVES SALAD,0,4,19,0,0,0,0,1,...,0,0,1,0,0,0,1,0,1,0
4,PNPPL CHNKS TS WN JC,PINEAPPLE CHUNKS ITS OWN JUICE,0,5,20,0,0,0,0,2,...,1,0,0,1,0,0,0,0,1,0


In [3]:
le = LabelEncoder()
y_transform = le.fit_transform(featured_df['y'].tolist())
y_transform.shape

(528610,)

In [4]:
gbdt = GradientBoostingClassifier()

In [5]:
## for now train with all x
## for now train with all y
all_cols = featured_df.columns.values.tolist()
all_cols.remove('x')
all_cols.remove('y')

train_x = featured_df[all_cols]
train_x.head()

,num_vowels,num_words,num_chars,num_digits,num_symbols,num_A,num_B,num_C,num_D,num_E,...,num_T,num_U,num_V,num_W,num_X,num_Y,num_Z,is_plural,has_char_repeat,has_color
0,0,4,20,0,0,0,1,2,4,0,...,0,0,0,0,1,0,0,0,1,0
1,0,2,13,0,0,0,1,0,0,0,...,2,0,1,0,0,0,0,0,1,0
2,0,1,3,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,4,19,0,0,0,0,1,2,0,...,0,0,1,0,0,0,1,0,1,0
4,0,5,20,0,0,0,0,2,0,0,...,1,0,0,1,0,0,0,0,1,0


In [6]:
train_x.shape

(528610, 34)

In [9]:
featured_df['label'] = y_transform
train_y = featured_df['label']
featured_df.head()

,x,y,num_vowels,num_words,num_chars,num_digits,num_symbols,num_A,num_B,num_C,...,num_U,num_V,num_W,num_X,num_Y,num_Z,is_plural,has_char_repeat,has_color,label
0,MXCN SHRDDD CHS BLND,MEXICAN SHREDDED CHEESE BLEND,0,4,20,0,0,0,1,2,...,0,0,0,1,0,0,0,1,0,58983
1,VGTBL SHRTNNG,VEGETABLE SHORTENING,0,2,13,0,0,0,1,0,...,0,1,0,0,0,0,0,1,0,100452
2,SQD,SQUID,0,1,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,88789
3,SLCD MNZNLL LVS SLD,SLICED MANZANILLA OLIVES SALAD,0,4,19,0,0,0,0,1,...,0,1,0,0,0,1,0,1,0,83595
4,PNPPL CHNKS TS WN JC,PINEAPPLE CHUNKS ITS OWN JUICE,0,5,20,0,0,0,0,2,...,0,0,1,0,0,0,0,1,0,70107


In [14]:
train_x = np.array(train_x)
train_y = np.array(train_y)

In [17]:
%%time
for i in range(0, len(train_x), 64):
    if i + 64 >= len(train_x) - 1:
        break
    else:
        gbdt.fit(train_x[i:i+64], train_y[i:i+64])

Wall time: 4h 13min 24s


In [30]:
shuffled = featured_df.sample(frac=1)
eval_df = shuffled.iloc[:1000]
preds = gbdt.predict(eval_df.drop(columns=['x', 'y', 'label'], axis=1))
preds

array([ 37224,  91653,  76161,  79588,  84761,  37224,  10146,  13622,
        37224,  21945,  91653,  97482,  12713,  10146,  12713,  71130,
        37224,  37224,  80747,  98679,  76161,  80747,  24673,  14220,
        37224,  21945,  84670,  12713,  10830,  79588,  37224,  37224,
        37224,  71130,  70897,  71130,  14220,  97482,  12713,  24673,
        91653, 105177,  12713,  65123,  12713,  80747,  24673,  24673,
        12914,  12713,  97482,  12713,  37224,  12914,  13622,  84670,
       105177,  24673,  71130, 103088,  37224,  68723,  79588,  65123,
        21945,  36543,  65123,  65123, 105177,  37224,  98679,  91653,
        37224,  43789,  21945,  24673,  12914,  58075,  12713,  24673,
        12713,  79588,  91653,  91653,  71130,  37224,  97482,  91653,
        51038,  21945,  14220,  24673,  12914,  12914,  68723,  21945,
        37224,   9706,  71130,  21945,  23053,  59541,  91653,  43789,
       105177,  12914,  14220,  37224,  71130,  91653,  83320,  12914,
      

In [32]:
eval_df['preds'] = le.inverse_transform(preds)

c:\users\kevin\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [34]:
eval_df[['x', 'y', 'preds']]

,x,y,preds
306263,SFT & CHW CKS,SOFT & CHEWY COOKIES,FROZEN RED RASPBERRIES
30951,SPLT TP WHT BRD,SPLIT TOP WHEAT BREAD,SWEET & SALTY
394372,RW CS RD UD,RAW CASHEWS ROASTED UNSALTED,RED RICE NOODLES
284640,D.. VL D MZR XTR VRG LV L,D.O.P. VAL DI MAZARA EXTRA VIRGIN OLIVE OIL,SALTED ALMOND ROLL
401501,PK & SP WN,PORK & SHRIMP WONTON,SMOOTH & SPICY PICANTE
327548,SD GI HS,SANDED GUMMI HEARTS,FROZEN RED RASPBERRIES
347692,BE SK BR,BITE SNACK BAR,BONELESS & SKINLESS CHICKEN BREAST
137222,BBQ FLD PTT CHS,BBQ FLAVORED POTATO CHIPS,CAKE SQUARE
361647,CE CT PS FS,CRINKLE CUT POTATOES FRIES,FROZEN RED RASPBERRIES
207455,TH CHL SC,THAI CHILI SAUCE,CLASSIC WHIPPED CREAM


In [19]:
le.inverse_transform([12])

array(['"DARN GOOD" CHILI MIX'], dtype='<U185')

In [ ]:
stacked = np.hstack((train_x, train_y))
np.random.shuffle(stacked)
train_num = int(0.8 * len(stacked))
train = stacked[:train_num]
test = stacked[train_num:]

train_x = train
## look at slicing